In [1]:
!pip install geopandas shapely

import geopandas as gpd
from shapely.geometry import Polygon, MultiPolygon
from shapely.ops import unary_union


!pip install -U geemap
import geemap, ee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.6 MB/s eta 0:00:00
  Attempting uninstall: earthengine-api
    Found existing installation: earthengine-api 1.5.24
    Uninstalling earthengine-api-1.5.24:
      Successfully uninstalled earthengine-api-1.5.24
  Attempting uninstall: geemap
    Found existing installation: geemap 0.35.3
    Uninstalling geemap-0.35.3:
      Successfully uninstalled geemap-0.35.3


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
bounded_box = '/content/drive/MyDrive/DL Map Files/bounded_box.kml'
thar = '/content/drive/MyDrive/DL Map Files/wwf_terr_ecos.kml'

In [20]:
from shapely.geometry import mapping, shape
from shapely.ops import transform
import pyproj

# Take your existing GeoDataFrame
geom = gdf.geometry.iloc[0]

# Define a transformer that drops Z
def drop_z(x, y, z=None):
    return (x, y)

# Apply the transformation
geom_2d = transform(drop_z, geom)

print(geom_2d.geom_type)   # should say 'Polygon'
print(geom_2d.has_z)       # should be False

# Convert to EE geometry
from shapely.geometry import mapping
import ee
ee.Authenticate()
ee.Initialize(project='static-retina-321611')

aoi = ee.Geometry(mapping(geom_2d))

Polygon
False


In [21]:
!pip install mss

In [24]:
Map = geemap.Map()
Map.addLayer(aoi, {}, "AOI (flattened)")
Map.centerObject(aoi)

# Define a buffer around the AOI for the export
buffered_aoi = aoi.buffer(10000)  # Buffer by 10,000 meters (10 km)

# Set the region for the map export
Map.region = buffered_aoi.bounds()

# Export the map as an image centered on the buffered AOI
Map.to_image(filename='/content/map_export.png')

print("Map exported as map_export.png")

# Convert gdf_wwf geometry to EE geometry and add to map
geom_wwf = gdf_wwf.geometry.iloc[0]
geom_wwf_2d = transform(drop_z, geom_wwf)
aoi_wwf = ee.Geometry(mapping(geom_wwf_2d))
Map.addLayer(aoi_wwf, {'color': 'brown'}, "WWF Ecoregion")

Map

$DISPLAY not set.
Map exported as map_export.png


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

#MODIS NDVI, EVI

In [ ]:
from datetime import datetime

start_date = '2000-02-18'
end_date = datetime.now().strftime('%Y-%m-%d')

modis = ee.ImageCollection('MODIS/061/MOD13A1') \
    .filterDate(start_date, end_date) \
    .select(['NDVI', 'EVI'])

# MODIS NDVI/EVI are scaled by 0.0001
scale_factor = 0.0001
modis = modis.map(lambda img: img.multiply(scale_factor).copyProperties(img, img.propertyNames()))

# --- Get MODIS native projection and scale ---
sample_img = modis.first()
modis_proj = sample_img.select('NDVI').projection()
modis_scale = modis_proj.nominalScale().getInfo()

print(f'MODIS nominal scale: {modis_scale} m')

# --- Create tiling grid for AOI ---
def make_grid(aoi, tile_size):
    bounds = aoi.bounds()
    lon_lat_list = ee.List(bounds.coordinates().get(0)).getInfo() # Evaluate server-side object

    lon_min = ee.Number(lon_lat_list[0][0])
    lat_min = ee.Number(lon_lat_list[0][1])
    lon_max = ee.Number(lon_lat_list[2][0])
    lat_max = ee.Number(lon_lat_list[2][1])

    lon_steps = lon_max.subtract(lon_min).divide(tile_size)
    lat_steps = lat_max.subtract(lat_min).divide(tile_size)

    lon_count = lon_steps.ceil().toInt()
    lat_count = lat_steps.ceil().toInt()

    grid = ee.FeatureCollection([])

    def make_row(lat_i):
        def make_tile(lon_i):
            x1 = lon_min.add(ee.Number(lon_i).multiply(tile_size))
            y1 = lat_min.add(ee.Number(lat_i).multiply(tile_size))
            x2 = x1.add(tile_size)
            y2 = y1.add(tile_size)
            geom = ee.Geometry.Rectangle([x1, y1, x2, y2])
            feat = ee.Feature(geom, {'tile_x': lon_i, 'tile_y': lat_i})
            return feat
        row = ee.List.sequence(0, lon_count.subtract(1)).map(make_tile)
        return ee.FeatureCollection(row)

    grid_list = ee.List.sequence(0, lat_count.subtract(1)).map(make_row)
    grid = ee.FeatureCollection(grid_list).flatten()
    return grid.filterBounds(aoi)

grid = make_grid(aoi, 0.1)  # roughly ~10km tiles (~0.1°)

# --- Sample mean NDVI/EVI for each tile per date ---
def sample_modis_per_tile(img):
    mean_vals = img.reduceRegions(
        collection=grid,
        reducer=ee.Reducer.mean(),
        scale=modis_scale,
        crs=modis_proj,
        tileScale=8
    )
    date = ee.Date(img.get('system:time_start')).format('YYYY_MM_dd')
    mean_vals = mean_vals.map(lambda f: f.set('date', date))
    return mean_vals

sampled = modis.map(sample_modis_per_tile).flatten()

# --- Flatten to feature collection for export ---
flat_fc = ee.FeatureCollection(sampled)

# --- Export to CSV (in Drive or Cloud Storage) ---
task = ee.batch.Export.table.toDrive(
    collection=flat_fc,
    description='MODIS_NDVI_EVI_per_tile',
    fileFormat='CSV'
)
task.start()

print("Export started: check Earth Engine Tasks tab.")

#DW with MODIS sizing

In [ ]:
# Dynamic World -> 8-day composites aligned to MODIS dates, downsampled to MODIS tiles
# Colab / Earth Engine Python API script
#
# Behavior:
# - For each MODIS MOD09A1 image date, build a DW 8-day mean composite (centered)
# - If no DW images in window, use latest prior DW image (carry-forward) if available
# - Aggregate DW probabilities (mean) per MODIS tile (same grid as MODIS script)
# - Export per-year CSVs to Drive
#
# NOTE: test on a small AOI / single year first.

import datetime, time
# ee.Initialize()

# --------------------------
# USER SETTINGS
# --------------------------
# Replace or reuse the exact AOI you used for MODIS
# Example placeholder (replace with your AOI FeatureCollection or Geometry)
# aoi = ee.FeatureCollection('projects/YOUR_PROJECT/assets/YOUR_AOI')  # <<-- update

# Year settings: export from this year to this year (set start_year <= end_year)
start_year = 2015            # DW starts mid-2015
end_year = datetime.datetime.now().year

# Tile/grid settings (must match your MODIS grid)
tile_size_deg = 0.1          # same tile size you used for MODIS
dw_bands = [                 # Dynamic World probability bands
    'water', 'trees', 'grass', 'flooded_vegetation',
    'crops', 'shrub_and_scrub', 'built', 'bare', 'snow_and_ice'
]

# Drive folder for exports
drive_folder = 'EarthEngineExports'

# --------------------------
# PREP: get MODIS dates & projection (used as temporal reference)
# --------------------------
# We'll use MODIS MOD09A1 (you used this earlier) to get the exact timestamps.
modis_for_dates = ee.ImageCollection("MODIS/061/MOD09A1").filterBounds(aoi)

# Filter out images missing time metadata
modis_for_dates = modis_for_dates.filter(ee.Filter.notNull(['system:time_start']))

# Projection & scale from MODIS (used in sampling)
sample_modis = ee.Image(modis_for_dates.first())
modis_proj = sample_modis.projection()
modis_scale = modis_proj.nominalScale()   # server-side Number

# Convert modis_proj and modis_scale to client values if needed for debug
print('MODIS projection:', modis_proj.getInfo())
print('MODIS nominal scale (m):', modis_scale.getInfo())

# --------------------------
# Build the same grid used for MODIS (tile_size_deg in degrees)
# --------------------------
def make_grid(aoi_fc, tile_size_deg=0.1):
    bounds = aoi_fc.bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    lon_min = ee.Number(ll.get(0))
    lat_min = ee.Number(ll.get(1))
    lon_max = ee.Number(ur.get(0))
    lat_max = ee.Number(ur.get(1))

    lon_steps = lon_max.subtract(lon_min).divide(tile_size_deg).ceil()
    lat_steps = lat_max.subtract(lat_min).divide(tile_size_deg).ceil()

    def make_row(lat_i):
        def make_tile(lon_i):
            x1 = lon_min.add(ee.Number(lon_i).multiply(tile_size_deg))
            y1 = lat_min.add(ee.Number(lat_i).multiply(tile_size_deg))
            x2 = x1.add(tile_size_deg)
            y2 = y1.add(tile_size_deg)
            geom = ee.Geometry.Rectangle([x1, y1, x2, y2])
            return ee.Feature(geom, {'tile_x': lon_i, 'tile_y': lat_i})
        row = ee.List.sequence(0, lon_steps.subtract(1)).map(make_tile)
        return ee.FeatureCollection(row)

    grid_list = ee.List.sequence(0, lat_steps.subtract(1)).map(make_row)
    grid_fc = ee.FeatureCollection(grid_list).flatten()
    return grid_fc.filterBounds(aoi_fc)

grid = make_grid(aoi, tile_size_deg)

# Add deterministic tile_id and centroids (optional but helpful)
def add_tile_info(f):
    f = ee.Feature(f)
    c = f.geometry().centroid(1)
    lon = ee.Number(c.coordinates().get(0))
    lat = ee.Number(c.coordinates().get(1))
    tid = ee.String('T_').cat(lon.format('%.6f')).cat('_').cat(lat.format('%.6f'))
    return f.set({'tile_id': tid, 'lon': lon, 'lat': lat})

grid = grid.map(add_tile_info)
print('Grid built (sample):', grid.limit(5).getInfo())

# --------------------------
# Load DW collection (probability bands)
# --------------------------
dw_coll = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
    .filterBounds(aoi) \
    .select(dw_bands)

# If DW collection empty for AOI (very unlikely), print warning
print('DW count (total):', dw_coll.size().getInfo())

# --------------------------
# Helper functions
# --------------------------
def dw_composite_for_modis(mod_img):
    """Build 8-day DW mean composite centered on MODIS image date.
       If no DW images in the 8-day window, use latest prior DW image (carry-forward).
       Returns an ee.Image (may be empty image with no bands)."""
    mod_img = ee.Image(mod_img)
    center = ee.Date(mod_img.get('system:time_start'))
    # 8-day window centered on modis date: [-3 days, +4 days] => 8 days total
    start = center.advance(-3, 'day')
    end = center.advance(4, 'day')

    dw_window = dw_coll.filterDate(start, end)
    dw_count = dw_window.size()

    # Prepare function to set default projection for each DW image (10 m)
    def prep(img):
        return ee.Image(img).setDefaultProjection('EPSG:4326', None, 10)
    prepared = dw_window.map(prep)

    # If we have images in window: take mean
    dw_mean_if = ee.ImageCollection(prepared).mean()

    # If there were no images in window, attempt to find the latest DW image before center
    # within a reasonable lookback (e.g., 60 days) to carry-forward probabilities
    lookback_days = 60
    prev_window = dw_coll.filterDate(center.advance(-lookback_days, 'day'), center)
    prev_prepared = prev_window.map(prep)
    prev_last = ee.Image(prev_prepared.sort('system:time_start', False).first())

    # Use conditional logic on dw_count
    dw_mean = ee.Algorithms.If(
        dw_count.gt(0),
        dw_mean_if.set('system:time_start', center.millis()),
        # else if prev_last exists (non-null), use prev_last (single image)
        ee.Algorithms.If(prev_last, ee.Image(prev_last).set('system:time_start', center.millis()), ee.Image().select())
    )

    return ee.Image(dw_mean)

def sample_dw_for_modis(mod_img):
    """For a given MODIS image, build DW composite and sample per tile.
       Returns a FeatureCollection with tile averages and date property."""
    mod_img = ee.Image(mod_img)
    date_str = ee.Date(mod_img.get('system:time_start')).format('YYYY_MM_dd')

    dw_img = dw_composite_for_modis(mod_img)
    dw_img = ee.Image(dw_img)  # cast

    bands = dw_img.bandNames()
    is_empty = bands.size().eq(0)

    # If empty, produce grid features with date but null probabilities
    no_data_fc = grid.map(lambda f: ee.Feature(f.geometry(), f.toDictionary()).set('date', date_str))

    # Otherwise, reduce DW probabilities to the grid (mean within each tile)
    def reduce_fn(img):
        return img.reduceRegions(
            collection=grid,
            reducer=ee.Reducer.mean(), # Add the reducer here
            scale=modis_scale,   # use MODIS scale (server-side)
            crs=modis_proj,
            tileScale=4,
            # bestEffort=True # Removed bestEffort as it's not a valid argument
        ).map(lambda f: ee.Feature(f.geometry(), f.toDictionary()).set('date', date_str))

    dw_by_tile_fc = ee.FeatureCollection(ee.Algorithms.If(is_empty, no_data_fc, reduce_fn(dw_img)))
    return dw_by_tile_fc

# --------------------------
# Build list of MODIS images to iterate by year
# --------------------------
# We'll use the same MODIS collection to get the exact timestamps
modis_dates_coll = modis_for_dates  # already defined above

# Optional: restrict modis_dates_coll to years >= 2015, because DW starts ~2015
modis_dates_coll = modis_dates_coll.filterDate(ee.Date.fromYMD(2015, 6, 1), ee.Date(datetime.datetime.now().strftime('%Y-%m-%d')))

# --------------------------
# Loop per year and export
# --------------------------
for year in range(max(start_year, 2015), end_year + 1):
    # define start/end of the year using ee.Date
    year_start = ee.Date.fromYMD(year, 1, 1)
    year_end = ee.Date.fromYMD(year, 12, 31)

    # Filter MODIS dates within the year
    modis_year = modis_dates_coll.filterDate(year_start, year_end)
    count_year = modis_year.size().getInfo()
    print(f'Year {year}: {count_year} MODIS dates to process.')

    if count_year == 0:
        print(f'No MODIS dates for year {year}; skipping.')
        continue

    # Map over modis_year -> for each modis date create DW sample per tile
    dw_samples_per_date = modis_year.map(sample_dw_for_modis).flatten()

    # Ensure DW band fields are present (force properties to the expected band names)
    def ensure_bands(feature):
        feature = ee.Feature(feature)
        # set each DW band to itself (keeps nulls if missing)
        for b in dw_bands:
            feature = feature.set(b, feature.get(b))
        return feature

    dw_samples_per_date = ee.FeatureCollection(dw_samples_per_date.map(ensure_bands))

    # Export this year's table to Drive
    desc = f'DW_8day_500m_aligned_{year}'
    print('Starting export:', desc)
    task = ee.batch.Export.table.toDrive(
        collection=dw_samples_per_date,
        description=desc,
        folder=drive_folder,
        fileNamePrefix=desc,
        fileFormat='CSV'
    )
    task.start()
    print('Export task started:', desc)
    # Optional: small pause to avoid API rate issues
    time.sleep(1)

#MODIS Raw

In [ ]:
import datetime

In [ ]:
start_year = 2001
end_year = datetime.datetime.now().year
tile_size_deg = 0.1     # ~10 km tiles
scale_factor = 0.0001

bands = [
    'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03',
    'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07'
]

# Load MODIS dataset
modis = ee.ImageCollection("MODIS/061/MOD09A1").select(bands)

# Projection info
sample_img = modis.first()
proj = sample_img.projection()
scale = proj.nominalScale().getInfo()
print(f"MODIS nominal scale: {scale} m")

# Build grid
def make_grid(aoi, tile_size_deg):
    bounds = aoi.bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    lon_min, lat_min = ee.Number(ll.get(0)), ee.Number(ll.get(1))
    lon_max, lat_max = ee.Number(ur.get(0)), ee.Number(ur.get(1))

    lon_steps = lon_max.subtract(lon_min).divide(tile_size_deg).ceil()
    lat_steps = lat_max.subtract(lat_min).divide(tile_size_deg).ceil()

    def make_row(lat_i):
        def make_tile(lon_i):
            x1 = lon_min.add(ee.Number(lon_i).multiply(tile_size_deg))
            y1 = lat_min.add(ee.Number(lat_i).multiply(tile_size_deg))
            x2 = x1.add(tile_size_deg)
            y2 = y1.add(tile_size_deg)
            geom = ee.Geometry.Rectangle([x1, y1, x2, y2])
            return ee.Feature(geom, {'tile_x': lon_i, 'tile_y': lat_i})
        return ee.FeatureCollection(ee.List.sequence(0, lon_steps.subtract(1)).map(make_tile))

    grid_fc = ee.FeatureCollection(ee.List.sequence(0, lat_steps.subtract(1)).map(make_row)).flatten()
    return grid_fc.filterBounds(aoi)

grid = make_grid(aoi, tile_size_deg)

# Sampling function
def sample_image(img):
    img_scaled = img.multiply(scale_factor)
    date_str = ee.Date(img.get('system:time_start')).format('YYYY_MM_dd')
    reduced = img_scaled.reduceRegions(
        collection=grid,
        reducer=ee.Reducer.mean(),
        scale=scale,
        crs=proj,
        tileScale=8
    ).map(lambda f: f.set('date', date_str))
    return reduced

# Keep band fields visible
def keep_bands(f):
    for b in bands:
        f = f.set(b, f.get(b))
    return f

# Loop per year and export
for year in range(start_year, end_year + 1):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)
    modis_year = modis.filterBounds(aoi).filterDate(start, end)
    sampled = modis_year.map(sample_image).flatten().map(keep_bands)

    desc = f"MODIS09A1_Reflectance_{year}"
    task = ee.batch.Export.table.toDrive(
        collection=sampled,
        description=desc,
        folder='EarthEngineExports',
        fileNamePrefix=desc,
        fileFormat='CSV'
    )
    task.start()
    print(f"✅ Export started for {year}")

In [ ]:
# ============================================================
# TEST SCRIPT — MODIS MOD09A1 Reflectance (1 month, small AOI)
# ============================================================


# Small AOI (example area in Sindh, Pakistan)
aoi = ee.Geometry.Polygon([
    [[68.1, 24.8],
     [68.3, 24.8],
     [68.3, 25.0],
     [68.1, 25.0]]
])

# Parameters
start_date = '2001-03-01'
end_date   = '2001-03-31'
scale_factor = 0.0001
tile_size_deg = 0.05   # ~5 km × 5 km tiles

bands = [
    'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b03',
    'sur_refl_b04', 'sur_refl_b05', 'sur_refl_b06', 'sur_refl_b07'
]

# Load MODIS MOD09A1 (8-day surface reflectance, 500 m)
modis = ee.ImageCollection("MODIS/061/MOD09A1") \
    .select(bands) \
    .filterDate(start_date, end_date) \
    .filterBounds(aoi)

# Get projection info
sample_img = modis.first()
proj = sample_img.projection()
scale = proj.nominalScale().getInfo()

print(f"MODIS nominal scale: {scale} m")

# Make a grid of small tiles
def make_grid(aoi, tile_size_deg):
    bounds = aoi.bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    lon_min, lat_min = ee.Number(ll.get(0)), ee.Number(ll.get(1))
    lon_max, lat_max = ee.Number(ur.get(0)), ee.Number(ur.get(1))

    lon_steps = lon_max.subtract(lon_min).divide(tile_size_deg).ceil()
    lat_steps = lat_max.subtract(lat_min).divide(tile_size_deg).ceil()

    def make_row(lat_i):
        def make_tile(lon_i):
            x1 = lon_min.add(ee.Number(lon_i).multiply(tile_size_deg))
            y1 = lat_min.add(ee.Number(lat_i).multiply(tile_size_deg))
            x2 = x1.add(tile_size_deg)
            y2 = y1.add(tile_size_deg)
            geom = ee.Geometry.Rectangle([x1, y1, x2, y2])
            return ee.Feature(geom, {'tile_x': lon_i, 'tile_y': lat_i})
        return ee.FeatureCollection(ee.List.sequence(0, lon_steps.subtract(1)).map(make_tile))

    grid_fc = ee.FeatureCollection(ee.List.sequence(0, lat_steps.subtract(1)).map(make_row)).flatten()
    return grid_fc.filterBounds(aoi)

grid = make_grid(aoi, tile_size_deg)

# Reduction function
def sample_image(img):
    img_scaled = img.multiply(scale_factor)
    date_str = ee.Date(img.get('system:time_start')).format('YYYY_MM_dd')
    reduced = img_scaled.reduceRegions(
        collection=grid,
        reducer=ee.Reducer.mean(),
        scale=scale,
        crs=proj,
        tileScale=8
    ).map(lambda f: f.set('date', date_str))
    return reduced

# Map and flatten
sampled = modis.map(sample_image).flatten()

# Ensure band names are kept
def keep_bands(f):
    for b in bands:
        f = f.set(b, f.get(b))
    return f

sampled = sampled.map(keep_bands)

# Export to Drive
task = ee.batch.Export.table.toDrive(
    collection=sampled,
    description='MODIS09A1_TestMonth',
    folder='EarthEngineExports',
    fileNamePrefix='MODIS09A1_TestMonth',
    fileFormat='CSV'
)
task.start()

print("✅ Export started for March 2001 test window.")


# Landsat

In [ ]:
# Cell A: Create monthly Landsat-9 composites (2022) and export per-month CSVs (table)

import datetime
import time

# --- Config (edit if needed) ---
year = 2022
start_year = year
end_year = year
drive_folder = 'EarthEngineExports'   # Drive folder where exports will be saved
tile_size_deg = 0.05                  # grid tile size (deg). Smaller -> more rows but finer spatial partition
cloud_pct_threshold = 40              # filter scenes with > this cloud %
bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']  # Landsat 9 SR bands
scale_factor = 0.0000275
offset = -0.2

# --- YOUR AOI must be defined in the notebook as 'aoi' (ee.Geometry or ee.FeatureCollection) ---
# Example (if you need to create a test AOI):
# aoi = ee.Geometry.Polygon([[[68.1,24.8],[68.3,24.8],[68.3,25.0],[68.1,25.0]]])

# ---- Helper functions ----
def make_grid(aoi_fc, tile_size_deg=0.05):
    bounds = aoi_fc.geometry().bounds()
    coords = ee.List(bounds.coordinates().get(0))
    ll = ee.List(coords.get(0))
    ur = ee.List(coords.get(2))
    lon_min = ee.Number(ll.get(0))
    lat_min = ee.Number(ll.get(1))
    lon_max = ee.Number(ur.get(0))
    lat_max = ee.Number(ur.get(1))

    lon_steps = lon_max.subtract(lon_min).divide(tile_size_deg).ceil()
    lat_steps = lat_max.subtract(lat_min).divide(tile_size_deg).ceil()

    def make_row(lat_i):
        def make_tile(lon_i):
            x1 = lon_min.add(ee.Number(lon_i).multiply(tile_size_deg))
            y1 = lat_min.add(ee.Number(lat_i).multiply(tile_size_deg))
            x2 = x1.add(tile_size_deg)
            y2 = y1.add(tile_size_deg)
            geom = ee.Geometry.Rectangle([x1, y1, x2, y2])
            return ee.Feature(geom, {'tile_x': lon_i, 'tile_y': lat_i})
        row = ee.List.sequence(0, lon_steps.subtract(1)).map(make_tile)
        return ee.FeatureCollection(row)

    grid_list = ee.List.sequence(0, lat_steps.subtract(1)).map(make_row)
    grid_fc = ee.FeatureCollection(grid_list).flatten()
    return grid_fc.filterBounds(aoi_fc)

# Build grid (server-side)
# Convert aoi Geometry to a FeatureCollection before passing to make_grid
aoi_fc = ee.FeatureCollection(aoi)
grid = make_grid(aoi_fc, tile_size_deg)

# Preprocess function for Landsat SR
def preprocess_landsat(img):
    # Select and scale bands
    scaled = img.select(bands).multiply(scale_factor).add(offset)
    return scaled.copyProperties(img, img.propertyNames())

# Create monthly date ranges for the year
months = [(m, datetime.date(year, m, 1)) for m in range(1,13)]

# Loop months: create median composite, sample grid, export CSV
for m, start_dt in months:
    start = ee.Date.fromYMD(year, m, 1)
    end = start.advance(1, 'month')
    # build collection and composite
    coll = (ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
            .filterBounds(aoi)
            .filterDate(start, end)
            .filter(ee.Filter.lt('CLOUD_COVER', cloud_pct_threshold))
            .map(preprocess_landsat))
    # If no images in month, skip
    count = coll.size().getInfo()
    if count == 0:
        print(f"Month {m:02d}/{year}: no Landsat9 images found; skipping.")
        continue

    composite = coll.median().clip(aoi).set('system:time_start', start.millis()).set('month', start.format('YYYY_MM'))
    # Reduce to grid (mean per tile)
    # Use reduceRegions with grid; this returns a FeatureCollection of tiles with mean band values
    reduced = composite.reduceRegions(
        collection=grid,
        reducer=ee.Reducer.mean(),
        scale=30,
        crs='EPSG:4326',
        tileScale=8,
    ).map(lambda f: ee.Feature(f.geometry(), f.toDictionary()).set('month', start.format('YYYY_MM')))

    desc = f"Landsat9_{year}_{m:02d}"
    filename = f"Landsat9_{year}_{m:02d}"
    print(f"Starting export: {desc} (tiles: approx {grid.size().getInfo()})  images={count}")
    task = ee.batch.Export.table.toDrive(
        collection=reduced,
        description=desc,
        folder=drive_folder,
        fileNamePrefix=filename,
        fileFormat='CSV'
    )
    task.start()
    # small pause to avoid firing too many tasks at once
    time.sleep(1)

print("All monthly export tasks started. Monitor the Tasks tab in the Earth Engine Code Editor or check Drive for file completion.")

Starting export: Landsat9_2022_01 (tiles: approx 24146)  images=58
Starting export: Landsat9_2022_02 (tiles: approx 24146)  images=77
Starting export: Landsat9_2022_03 (tiles: approx 24146)  images=55
Starting export: Landsat9_2022_04 (tiles: approx 24146)  images=76
Starting export: Landsat9_2022_05 (tiles: approx 24146)  images=83
Starting export: Landsat9_2022_06 (tiles: approx 24146)  images=53
Starting export: Landsat9_2022_07 (tiles: approx 24146)  images=25
Starting export: Landsat9_2022_08 (tiles: approx 24146)  images=17
Starting export: Landsat9_2022_09 (tiles: approx 24146)  images=67
Starting export: Landsat9_2022_10 (tiles: approx 24146)  images=80
Starting export: Landsat9_2022_11 (tiles: approx 24146)  images=78
Starting export: Landsat9_2022_12 (tiles: approx 24146)  images=75
All monthly export tasks started. Monitor the Tasks tab in the Earth Engine Code Editor or check Drive for file completion.


preprocess to DW CSVs

In [ ]:
# Cell B: Postprocess monthly CSVs into DW-ready CSVs (adds placeholders),
# run after exports complete and files appear in Drive.

import pandas as pd
import numpy as np
import glob, os

data_folder = '/content/drive/MyDrive/EarthEngineExports/'   # same folder used for exports
output_folder = '/content/drive/MyDrive/EarthEngineExports/DW_ready/'
os.makedirs(output_folder, exist_ok=True)

# Pattern for monthly exports we created: Landsat9_2022_01.csv ...
csv_files = sorted(glob.glob(os.path.join(data_folder, 'Landsat9_2022_*.csv')))
print(f"Found {len(csv_files)} exported monthly CSVs in Drive.")

for f in csv_files:
    print(f"⚙️ Processing {os.path.basename(f)}...")
    df = pd.read_csv(f)
    # Some GEE exports add columns like 'tile_x','tile_y','month', etc.
    # We only require the reflectance bands (SR_B2..SR_B7) and may keep tile_x/tile_y or .geo for location.
    bands = ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7']
    # keep present columns (avoid KeyError)
    keep_cols = [c for c in bands if c in df.columns]
    # keep tile id/geometry if available so you can map results back
    for extra in ['tile_x','tile_y','system:index','.geo','month','date']:
        if extra in df.columns:
            keep_cols.append(extra)
    df_small = df[keep_cols].copy()

    # Add placeholders for missing Sentinel-2 bands expected by DW:
    # We'll provide columns: B1,B2,B3,B4,B5,B6,B7,B8A,B11,B12 (DW uses 9 bands but including B1 is optional)
    # Map Landsat SR_B2..SR_B7 to B2..B7 respectively, fill others with zeros.
    df_proc = pd.DataFrame()
    # Map Landsat->S2 names if present
    mapping = {'SR_B2':'B2','SR_B3':'B3','SR_B4':'B4','SR_B5':'B5','SR_B6':'B6','SR_B7':'B7'}
    for s2band in ['B1','B2','B3','B4','B5','B6','B7','B8A','B11','B12']:
        if s2band in ['B2','B3','B4','B5','B6','B7']:
            # original Landsat name
            ls_name = [k for k,v in mapping.items() if v==s2band][0]
            if ls_name in df_small.columns:
                df_proc[s2band] = df_small[ls_name].astype(float)
            else:
                df_proc[s2band] = 0.0
        else:
            # fill placeholders
            df_proc[s2band] = 0.0

    # Bring along tile_x/tile_y/.geo/month if present
    for col in ['tile_x','tile_y','system:index','.geo','month','date']:
        if col in df_small.columns:
            df_proc[col] = df_small[col]

    # Clip values to [0,1]
    for b in ['B1','B2','B3','B4','B5','B6','B7','B8A','B11','B12']:
        df_proc[b] = np.clip(df_proc[b], 0.0, 1.0)

    # Save per-month DW-ready CSV
    base = os.path.basename(f).replace('.csv','')
    out_path = os.path.join(output_folder, f"{base}_dw_ready.csv")
    df_proc.to_csv(out_path, index=False)
    print(f"✅ Saved DW-ready file: {out_path}")

print("All available monthly CSVs processed to DW-ready files.")


# Desert Area
Load the KML file from `/content/wwf_terr_ecos.kml` into a GeoDataFrame, calculate the area of the geometries in this new GeoDataFrame and compare it with the area of the geometries in the GeoDataFrame loaded from `/content/bounded_box.kml`.

**Reasoning**:
Load the KML file specified by `kml2` into a new GeoDataFrame, then print its shape, geometry type counts, and the first few rows to verify the loading and content.



In [3]:
gdf_wwf = gpd.read_file(thar, driver='KML')

print(gdf_wwf.shape)
print(gdf_wwf.geometry.type.value_counts())
gdf_wwf.head()

(1, 3)
Polygon    1
Name: count, dtype: int64


,Name,Description,geometry
0,,,"POLYGON ((71.08521 24.69305, 71.08553 24.6845,..."


**Reasoning**:
The previous steps successfully loaded the KML file into `gdf_wwf` and displayed its initial information. Now, to fulfill the main task, calculate the area of the geometries in both `gdf` and `gdf_wwf` and then compare them. First, it's good practice to display `gdf` again to remember its contents before proceeding with area calculation.



In [5]:
gdf = gpd.read_file(bounded_box, driver='KML')

print('Original GeoDataFrame (gdf) from bounded_box.kml:')
print(gdf.shape)
print(gdf.geometry.type.value_counts())
gdf.head()

Original GeoDataFrame (gdf) from bounded_box.kml:
(1, 3)
Polygon    1
Name: count, dtype: int64


,Name,Description,geometry
0,Buffered Bounding Box,,"POLYGON Z ((67.84825 23.71241 0, 76.61425 23.7..."


**Reasoning**:
To accurately compare the areas of the geometries, both GeoDataFrames need to be reprojected to a common projected coordinate system (PCS) that preserves area, such as a suitable UTM zone. Then, calculate and print the area of each geometry in square kilometers and compare them.



In [6]:
print('Calculating areas...')

# Define a suitable projected CRS for area calculation (e.g., UTM zone for the region)
# Given the coordinates are in India (around 23-24 N, 71-76 E), UTM Zone 43N (EPSG:32643) is a good choice.
projected_crs = 'EPSG:32643'

# Calculate area for gdf
gdf_proj = gdf.to_crs(projected_crs)
area_gdf_sqkm = gdf_proj.geometry.area.sum() / 1_000_000 # Convert m^2 to km^2
print(f'Area of gdf (from bounded_box.kml): {area_gdf_sqkm:.2f} sq km')

# Calculate area for gdf_wwf
gdf_wwf_proj = gdf_wwf.to_crs(projected_crs)
area_gdf_wwf_sqkm = gdf_wwf_proj.geometry.area.sum() / 1_000_000 # Convert m^2 to km^2
print(f'Area of gdf_wwf (from wwf_terr_ecos.kml): {area_gdf_wwf_sqkm:.2f} sq km')

# Compare the areas
if area_gdf_sqkm > area_gdf_wwf_sqkm:
    comparison = 'larger than'
elif area_gdf_sqkm < area_gdf_wwf_sqkm:
    comparison = 'smaller than'
else:
    comparison = 'equal to'

print(f'The area of the geometry in bounded_box.kml is {comparison} the area of the geometry in wwf_terr_ecos.kml.')

Calculating areas...
Area of gdf (from bounded_box.kml): 656428.46 sq km
Area of gdf_wwf (from wwf_terr_ecos.kml): 238781.66 sq km
The area of the geometry in bounded_box.kml is larger than the area of the geometry in wwf_terr_ecos.kml.


In [15]:
print(area_gdf_wwf_sqkm/area_gdf_sqkm*100)

36.37588522020637
